# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#pindah ke direktori yang digunakan
%cd '/content/drive/MyDrive/1 Pengajaran Pendidikan/Kuliah/SI_BISAAI_NLP'

[Errno 2] No such file or directory: '/content/drive/MyDrive/1 Pengajaran Pendidikan/Kuliah/SI_BISAAI_NLP'
/content


#1.Import Library




In [ ]:
 ! pip install dataframe-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.6 MB 14.7 MB/s 


In [ ]:
!pip install varname

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras-metrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 11.9 MB/s 


In [ ]:
import nltk
import pandas as pd
import re
import tensorflow as tf
import time
import multiprocessing
import io
import gensim
import numpy as np
import matplotlib.pyplot as plt
import keras_metrics as km
import pickle
import keras


from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
#from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from varname import nameof


nltk.download('punkt')
from nltk.tokenize import word_tokenize


from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#2.Load Data

In [ ]:
#simpan path dataset
path_data = "ulasan-20.csv"

#read dataset
data_dataset = pd.read_csv(path_data, sep=",", header=[0], encoding="UTF-8")
print(data_dataset)

FileNotFoundError: ignored

#3.Implementasi Preprocessing

##3.1. Casefolding

In [ ]:
#mengambil data ulasan
data_content = data_dataset ['ulasan']
print(data_content[:4])

In [ ]:
#casefolding
data_casefolding = data_content.str.lower()
data_casefolding.head()

##3.2. Filtering/Removing

```
# This is formatted as code
```



In [ ]:
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|
\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))\n''',
 " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub(r'[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  
#hapus simbol'[!#?,.:";@()-_/\']'
#  hapus tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)


In [ ]:
data_filtering

In [ ]:
#tokenization
data_tokens = [word_tokenize(line) for line in data_filtering]
print(data_tokens)

##3.4. Konversi Slangword

## 3.5. Stopword Removal

In [ ]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]

In [ ]:
print(pd.DataFrame(default_stop_words)[:10])

In [ ]:
len(data_notstopword)

In [ ]:
data_notstopword

##3.6. Stemming

In [ ]:
 #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i
white_list = ["bali"]
factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      if(word not in white_list):
        word = ind_stemmer.stem(word)
      if(len(word)>3):
        temp.append(word)
    return temp

reviews = [stemmer (line) for line in data_notstopword]
print(reviews)

In [ ]:
#coba ganti ke pandas
data_bersih = pd.Series(reviews)

In [ ]:
#coba simpan ke csv
data_bersih.to_csv('databersih.csv')

#4.Konversi Kalimat

In [ ]:
#Pembuatan Kamus kata
t  = Tokenizer()
fit_text = reviews
t.fit_on_texts(fit_text)

#Pembuatan Id masing-masing kata
sequences = t.texts_to_sequences(reviews)

#hapus duplikat kata yang muncul
list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

#mencari max length sequence
def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
    return maxList, maxLength 
      
# Driver Code 
max_seq, max_length_seq = FindMaxLength(list_set_sequence)
jumlah_index = len(t.word_index) +1

print('jumlah index : ',jumlah_index,'\n')
print('word_index : ',t.word_index,'\n')
print('index kalimat asli     : ', sequences,'\n')
print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
print('panjang max kalimat : ', max_length_seq,'kata','\n')
# print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

count_word = [len(i) for i in list_set_sequence]
print('list panjang kalimat : ', count_word)
max_len_word = max(count_word)
print(max_len_word)

In [ ]:
min_len_word=min(count_word)
print(min_len_word)

In [ ]:
print ("Original list is : " + str(list_set_sequence[0])) 

## 4.1. Padding

In [ ]:
#Padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))],
maxlen= max_len_word, padding='pre')
padding[:10]

# TOPIC 5 - FEATURE EXTRACTION

##Load Hasil Preprocessing & Join


In [ ]:

reviews2 = [" ".join(r) for r in reviews]


##Convectorizer

In [ ]:
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(reviews2)
print(vectorizer.vocabulary_ )
print(X_vec.todense())

##Bag of Word 

### Building the vocabulary

In [ ]:
set_of_words = set()
for sentence in reviews2:
    for word in sentence.split():
        set_of_words.add(word)
vocab = list(set_of_words)
print(vocab)

### Fetching the position of each word in the vocabulary

In [ ]:
position = {}
for i, token in enumerate(vocab):
    position[token] = i
print(position)

### Creating a matrix to hold the Bag of Words representation

In [ ]:
bow_matrix = np.zeros((len(reviews2), len(vocab)))

In [ ]:
for i, preprocessed_sentence in enumerate(reviews2):
    for token in preprocessed_sentence.split():   
        bow_matrix[i][position[token]] = bow_matrix[i][position[token]] + 1

In [ ]:
bow_matrix

##N-gram

### Unigram, Bigram, Trigram

In [ ]:
vectorizer_ngram_range = CountVectorizer(analyzer='word', ngram_range=(1,3))
bow_matrix_ngram = vectorizer_ngram_range.fit_transform(reviews2)

In [ ]:
print(vectorizer_ngram_range.get_feature_names())
print(bow_matrix_ngram.toarray())

### Max_feature

In [ ]:
vectorizer_max_features = CountVectorizer(analyzer='word', ngram_range=(1,3), max_features = 100)
bow_matrix_max_features = vectorizer_max_features.fit_transform(reviews2)

In [ ]:
print(vectorizer_max_features.get_feature_names())
print(bow_matrix_max_features.toarray())

### Max_df & Min_df

In [ ]:
vectorizer_max_min = CountVectorizer(analyzer='word', ngram_range=(1,3), max_df =3, min_df = 2)
bow_matrix_max_min = vectorizer_max_min.fit_transform(preprocessed_corpus)

In [ ]:
print(vectorizer_max_min.get_feature_names())
print(bow_matrix_max_min.toarray())

## TF-IDF

In [ ]:
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(preprocessed_corpus)

In [ ]:
print(vectorizer.get_feature_names())
print(tf_idf_matrix.toarray())
print("\nThe shape of the TF-IDF matrix is: ", tf_idf_matrix.shape)